In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import Birch
import random

In [2]:
print(os.getcwd())

D:\GitCloneProject\Stomach-Status-Classification\Data Preprocessing\bag_of_visual_word


In [3]:
os.chdir("..")
os.chdir("..")
print(os.getcwd())

D:\GitCloneProject\Stomach-Status-Classification


In [4]:
main_data_dir = os.getcwd() + "\\Data set"
kmean_data_dir = main_data_dir + "\\spc_dataset"
import glob

In [5]:
from sklearn import metrics
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [6]:
descriptor = 'sift'

data = main_data_dir + "\\{}".format(descriptor)
print(data)

D:\GitCloneProject\Stomach-Status-Classification\Data set\sift


In [7]:
data_train = [i for i in glob.glob(data + '\\*') if 'train' in i]
data_test = [i for i in glob.glob(data + '\\*') if 'test' in i]

print(data_train)
print(data_test)

['D:\\GitCloneProject\\Stomach-Status-Classification\\Data set\\sift\\sift_train_300_3_0.04_10_1.6.json']
['D:\\GitCloneProject\\Stomach-Status-Classification\\Data set\\sift\\sift_test_300_3_0.04_10_1.6.json']


In [8]:
import json 
def load_json(path):
    f = open(path)
    return  json.load(f)

train_json = load_json(data_train[0])
test_json = load_json(data_test[0])

In [9]:
save_train_path = main_data_dir + "\\spc_dataset\\{0}cv_{1}\\" + data_train[0].split("\\")[-1][:-5] + "histo_{2}_cv_index{3}.csv"
save_test_path = main_data_dir + "\\spc_dataset\\{0}cv_{1}\\" + data_test[0].split("\\")[-1][:-5] + "histo_{2}_cv_index{3}.csv"
print(save_train_path.format(10, 128, 128, 0))

D:\GitCloneProject\Stomach-Status-Classification\Data set\spc_dataset\10cv_128\sift_train_300_3_0.04_10_1.6histo_128_cv_index0.csv


In [15]:
def chunks(lst, n):
    random.shuffle(lst) 
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def make_data(k_choosed = 128, random_state = 42, cv = 10, test_size = 2,
              json_data = [], train_path = None, test_path = None):
    train_json = json_data[0]
    test_json = json_data[1]
    
    train_nor_js = train_json["Group 1 - Normal"]
    train_ab_js = train_json["Group 2 - Abnormal"]
    
    test_nor_js = test_json["Group 1 - Normal"]
    test_ab_js = test_json["Group 2 - Abnormal"]
    
    ab_vector_lst = train_ab_js + test_ab_js
    nor_vector_lst = train_nor_js + test_nor_js
    
    ab_folds = list(chunks(ab_vector_lst, round(len(ab_vector_lst)/cv)))
    nor_folds = list(chunks(nor_vector_lst, round(len(nor_vector_lst)/cv)))
    
    folds = [x + y for x, y in zip(ab_folds, nor_folds)]
    
    for x in range(len(folds)):
        print("Starting Fold : {}".format(x))
        print("\tData Extraction from Fold")
        test_filenames = folds[x]
        train_start = []
        train_end = []
        if folds[:x]:
            for i in folds[:x]:
                train_start += i 
        if folds[x+1:]:
            for i in folds[x+1:]:
                train_end += i 
        train_filenames = train_start + train_end
        
        concat_lst = []
        print("\tFinish Data Extraction from Fold")
        print("\tData Preparing")
        for img_dict in train_filenames:
            for key in img_dict:
                img_vector_lst = img_dict[key]
                img_vector = np.array(img_vector_lst)
                concat_lst.append(img_vector)
        
        data_cluster = concat_lst[0]
        for i in range(1, len(concat_lst)):
            concat_data = np.concatenate((data_cluster, concat_lst[i]), axis=0)
            data_cluster = concat_data
        print("\tFinish Data Preparing for Model")
        print("\tModel Training Data Extraction")
        save_dict = {i : [] for i in range(k_choosed+2)}
        model = Birch(n_clusters=k_choosed).fit(data_cluster)
        
        for img_dict in train_filenames:
            for key in img_dict:
                img_vector_lst = img_dict[key]
                img_vector = np.array(img_vector_lst)
                km_vector = model.predict(np.array(img_vector))
                his_vector = np.histogram(km_vector, bins = k_choosed, density = True)[0]
                save_dict[0].append(key)
                for j in range(1, k_choosed+1):
                    save_dict[j].append(his_vector[j-1])
                if 'Normal' in key:
                    save_dict[k_choosed+1].append(0)
                else:
                    save_dict[k_choosed+1].append(1)
        
        if train_path:
            df = pd.DataFrame(save_dict)
            df.to_csv(train_path.format(cv, k_choosed, k_choosed, x))
            
        print("\tModel Testing Data Extraction")
        save_dict = {i : [] for i in range(k_choosed+2)}
        for img_dict in test_filenames:
            for key in img_dict:
                img_vector_lst = img_dict[key]
                img_vector = np.array(img_vector_lst)
                km_vector = model.predict(np.array(img_vector))
                his_vector = np.histogram(km_vector, bins = k_choosed, density = True)[0]
                save_dict[0].append(key)
                for j in range(1, k_choosed+1):
                    save_dict[j].append(his_vector[j-1])
                if 'Normal' in key:
                    save_dict[k_choosed+1].append(0)
                else:
                    save_dict[k_choosed+1].append(1)
        
        if test_path:
            df = pd.DataFrame(save_dict)
            df.to_csv(test_path.format(cv, k_choosed, k_choosed, x))
        print("\tFinish Data Extraction")

In [16]:
make_data(cv = 10, k_choosed = 256, test_size = 2,
          json_data = [train_json, test_json], 
          train_path = save_train_path, 
          test_path = save_test_path)

Starting Fold : 0
	Data Extraction from Fold
	Finish Data Extraction from Fold
	Data Preparing
	Finish Data Preparing for Model
	Model Training Data Extraction
	Model Testing Data Extraction
	Finish Data Extraction
Starting Fold : 1
	Data Extraction from Fold
	Finish Data Extraction from Fold
	Data Preparing
	Finish Data Preparing for Model
	Model Training Data Extraction
	Model Testing Data Extraction
	Finish Data Extraction
Starting Fold : 2
	Data Extraction from Fold
	Finish Data Extraction from Fold
	Data Preparing
	Finish Data Preparing for Model
	Model Training Data Extraction
	Model Testing Data Extraction
	Finish Data Extraction
Starting Fold : 3
	Data Extraction from Fold
	Finish Data Extraction from Fold
	Data Preparing
	Finish Data Preparing for Model
	Model Training Data Extraction
	Model Testing Data Extraction
	Finish Data Extraction
Starting Fold : 4
	Data Extraction from Fold
	Finish Data Extraction from Fold
	Data Preparing
	Finish Data Preparing for Model
	Model Train